In [2]:
import time
import math

import numpy as np


n = 100
size = 1000
x = np.random.random_sample(size)
y = np.random.random_sample(size)
#ngrid = 33

## R's bcv & kde2d

In [12]:
import rpy2.robjects as ro
from rpy2.robjects.numpy2ri import numpy2ri
ro.conversion.py2ri = numpy2ri
from rpy2.robjects.packages import importr
mass = importr('MASS')

for p in range(2, 10):
    ngrid = math.pow(2, p) + 1
    t0 = time.time()
    for i in range(n):
        x = np.random.random_sample(size)
        y = np.random.random_sample(size)
        bandwidth_x = np.asarray(mass.bcv(x)[0])
        bandwidth_y = np.asarray(mass.bcv(y)[0])
        kde = mass.kde2d(x, y, np.asarray([bandwidth_x, bandwidth_y]), n=np.asarray([ngrid]))
    print(int(ngrid), '\t', time.time() - t0)

5 	 0.6156280040740967
9 	 0.6511189937591553
17 	 0.7998111248016357
33 	 0.9960660934448242
65 	 1.423185110092163
129 	 2.1830689907073975
257 	 3.57030987739563
513 	 6.9181602001190186


## fastKDE

In [13]:
from fastkde import fastKDE

for p in range(2, 10):
    ngrid = math.pow(2, p) + 1
    t0 = time.time()
    for i in range(n):
        x = np.random.random_sample(size)
        y = np.random.random_sample(size)
        pdf, axes = fastKDE.pdf(x, y, numPoints=ngrid)
    print(int(ngrid), '\t', time.time() - t0)

5 	 0.6640889644622803
9 	 0.6731390953063965
17 	 0.7362771034240723
33 	 0.907505989074707
65 	 1.202699899673462
129 	 2.406049966812134
257 	 16.08696699142456
513 	 28.61909294128418


## KDEMultivariate
Higher overhead. Can specify bandwidth. Slowest.

In [10]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate

t0 = time.time()
for i in range(n):
    x = np.random.random_sample(size)
    y = np.random.random_sample(size)
    kde = KDEMultivariate([x, y], var_type='cc', bw=[0.1, 0.1])
    kde.pdf()
time.time() - t0

7.120078086853027

## gaussian_kde
Faster. Can't specify bandwidth.

In [ ]:
from scipy.stats import gaussian_kde

t0 = time.time()
for i in range(n):
    x = np.random.random_sample(size)
    y = np.random.random_sample(size)
    kde = gaussian_kde([x, y], bw_method='silverman')
    kde.pdf(np.random.random_sample((2, 25)))
time.time() - t0